<a href="https://colab.research.google.com/github/sathvik044/ML-ASS4-Q1/blob/main/ML_ASS4_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('Enjoy sports.csv')

# Display shape and first few rows of the dataset
print(df.shape)
print(df.head())

# Function to find unique values in an attribute
def values(attr):
    l = []
    for x in attr:
        if x not in l:
            l.append(x)
    return l

# Function to calculate entropy
def Entropy(data):
    d = data.iloc[:, -1]  # The target column
    d = d.value_counts()
    s = 0
    for v in d.keys():
        p = d[v] / sum(d)
        s -= p * np.log2(p)
    return s

# Function to calculate Information Gain
def IG(data, A):
    Es = Entropy(data)  # Entropy of the whole dataset
    val = values(data[A])  # Unique values of the feature
    s_c = data[A].value_counts()  # Value counts of the feature
    s_v = []

    for v in range(len(val)):
        ds = data[data[A] == val[v]]  # Subset where the feature equals a particular value
        s = 0
        for res in values(data.iloc[:, -1]):  # Iterate over all possible results
            try:
                pi = ds.iloc[:, -1].value_counts()[res] / len(ds)
                s -= pi * np.log2(pi)
            except:
                s = 0
        s_v.append(s)

    for i in range(len(val)):
        Es = Es - s_c[val[i]] * s_v[i] / sum(s_c)  # Weighted sum of entropy for all splits

    return Es

# Node class to represent each node in the tree
class Node():
    def _init_(self, name=None, attr=None):
        self.name = name
        self.attr = attr

    def call_(self):
        return self.name

# Function to create a node by finding the best feature using Information Gain
def DTNode(data, features_used):
    node = Node()
    IGmax = 0
    v_best = None
    val_list = [v for v in values(data)[:-1] if v not in features_used]

    if val_list != []:
        for v in val_list:
            if IG(data, v) > IGmax:
                IGmax = IG(data, v)
                v_best = v

        if v_best:
            features_used.append(v_best)
            node.name = v_best
            node.attr = values(data[v_best])

            return node
        else:
            return None

    return None

# Recursive function to build the decision tree
def DTclassifier(data, features_used):
    root = DTNode(data, features_used)

    DT_dict = {}
    if root != None:
        item = []
        for attr in root.attr:
            dataN = data[data[root.name] == attr]
            if Entropy(dataN) == 0:  # If entropy is 0, it's a leaf node
                item.append((attr, values(dataN.iloc[:, -1])[0]))
            else:
                dt = DTclassifier(dataN, features_used)  # Recursive call
                item.append((attr, dt))
        DT_dict[root.name] = item
    return DT_dict

# Fix: Call the classifier and print the decision tree
features_used = []
decision_tree = DTclassifier(df, features_used)
print("Decision Tree:", decision_tree)

(14, 6)
   Day   Outlook Temp. Humidity    Wind Decision
0    1     Sunny   Hot     High    Weak       No
1    2     Sunny   Hot     High  Strong       No
2    3  Overcast   Hot     High    Weak      Yes
3    4      Rain  Mild     High    Weak      Yes
4    5      Rain  Cool   Normal    Weak      Yes
Decision Tree: {'Day': [(1, 'No'), (2, 'No'), (3, 'Yes'), (4, 'Yes'), (5, 'Yes'), (6, 'No'), (7, 'Yes'), (8, 'No'), (9, 'Yes'), (10, 'Yes'), (11, 'Yes'), (12, 'Yes'), (13, 'Yes'), (14, 'No')]}
